# normal feed forward

## PROSTATE + BLADDER + BREAST 

In [1]:
# Jupyter Notebook version of main.py

import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
from torch.utils.data import random_split, ConcatDataset, DataLoader

# Import your dataset and models
from dataloader import HistopathologyDataset
from model import Attention, GatedAttention

# ---------------------------
# Set parameters manually:
# ---------------------------
epochs = 20
lr = 0.0005
reg = 1e-4
# Use your actual paths to the folder (or file) containing your .pt embeddings 
# and the CSV file with labels.
# (Note: if your HistopathologyDataset expects a directory, make sure you supply that.)
# Paths for Cancer Prostate
embeddings_dir_prostate = "/data/temporary/amirhosein/model_script_prostate/pt_files/"
labels_csv_prostate = "/data/temporary/amirhosein/Prostate_Project/prostate_tp53_labels.csv"

# Paths for Cancer Bladder
embeddings_dir_bladder = "/data/temporary/amirhosein/model_script_bladder/pt_files/"
labels_csv_bladder = "/data/temporary/amirhosein/bladder_project/bladder_tp53_labels.csv"

# Paths for Cancer Breast
embeddings_dir_breast = "/data/temporary/amirhosein/modelscript_breast/pt_files/"
labels_csv_breast = "/data/temporary/amirhosein/breast_Project(subdirectories)/breast_tp53_labels.csv"

input_dim = 1024
model_type = "attention"   # Choose either "attention" or "gated_attention"
no_cuda = False

cuda = (not no_cuda) and torch.cuda.is_available()

# ---------------------------
# Set seeds and print status:
# ---------------------------
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

# ---------------------------
# Create dataset objects.
# ---------------------------
dataset_prostate = HistopathologyDataset(embeddings_dir_prostate, labels_csv_prostate)
dataset_bladder = HistopathologyDataset(embeddings_dir_bladder, labels_csv_bladder)
dataset_breast = HistopathologyDataset(embeddings_dir_breast, labels_csv_breast)


# Suppose you want 80% of Cancer C for training and 20% for testing.
total_prostate = len(dataset_prostate)
train_size_prostate = int(0.8 * total_prostate)
test_size_prostate  = total_prostate - train_size_prostate

dataset_prostate_train, dataset_prostate_test = random_split(dataset_prostate, [train_size_prostate, test_size_prostate])

train_dataset = ConcatDataset([dataset_breast, dataset_bladder, dataset_prostate_train])
test_dataset  = dataset_prostate_test  # held-out portion from Cancer C

loader_kwargs = {'num_workers': 0, 'pin_memory': True} if cuda else {}
train_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, **loader_kwargs)
test_loader  = data.DataLoader(test_dataset, batch_size=1, shuffle=False, **loader_kwargs)

# ---------------------------
# Initialize the Model:
# ---------------------------
print('Initializing Model')
if model_type == 'attention':
    model = Attention(input_dim=input_dim)
elif model_type == 'gated_attention':
    model = GatedAttention(input_dim=input_dim)
else:
    raise ValueError("Unknown model type. Choose 'attention' or 'gated_attention'.")

if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=reg)

# ---------------------------
# Define training and testing functions:
# ---------------------------
def train(epoch):
    model.train()
    train_loss = 0.0
    train_error = 0.0
    for batch_idx, (data_batch, label) in enumerate(train_loader):
        if cuda:
            data_batch, label = data_batch.cuda(), label.cuda()
        data_batch, label = Variable(data_batch), Variable(label)
    
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data_batch, label)
        train_loss += loss.item()
        error, _ = model.calculate_classification_error(data_batch, label)
        train_error += error
    
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print('Epoch: {}, Loss: {:.4f}, Train Error: {:.4f}'.format(epoch, train_loss, train_error))
    
def test():
    model.eval()
    test_loss = 0.0
    test_error = 0.0
    with torch.no_grad():
        for batch_idx, (data_batch, label) in enumerate(test_loader):
            if cuda:
                data_batch, label = data_batch.cuda(), label.cuda()
            data_batch, label = Variable(data_batch), Variable(label)
            loss, attention_weights = model.calculate_objective(data_batch, label)
            test_loss += loss.item()
            error, predicted_label = model.calculate_classification_error(data_batch, label)
            test_error += error
    
            if batch_idx < 5:
                print('\nTrue Bag Label: {}  Predicted Bag Label: {}'
                      .format(label.cpu().numpy()[0], predicted_label.cpu().numpy()[0]))
    
    test_loss /= len(test_loader)
    test_error /= len(test_loader)
    print('\nTest Set, Loss: {:.4f}, Test Error: {:.4f}'.format(test_loss, test_error))
    
# ---------------------------
# Run training and testing:
# ---------------------------
print('Start Training')
for epoch in range(1, epochs + 1):
    train(epoch)
print('Start Testing')
test()


GPU is ON!
Initializing Model
Start Training
Epoch: 1, Loss: 0.5433, Train Error: 0.2603
Epoch: 2, Loss: 0.4700, Train Error: 0.2148
Epoch: 3, Loss: 0.4248, Train Error: 0.1863
Epoch: 4, Loss: 0.3851, Train Error: 0.1629
Epoch: 5, Loss: 0.3614, Train Error: 0.1622
Epoch: 6, Loss: 0.3227, Train Error: 0.1430
Epoch: 7, Loss: 0.2790, Train Error: 0.1195
Epoch: 8, Loss: 0.2601, Train Error: 0.1060
Epoch: 9, Loss: 0.2166, Train Error: 0.0903
Epoch: 10, Loss: 0.1890, Train Error: 0.0818
Epoch: 11, Loss: 0.1667, Train Error: 0.0676
Epoch: 12, Loss: 0.1390, Train Error: 0.0576
Epoch: 13, Loss: 0.1084, Train Error: 0.0427
Epoch: 14, Loss: 0.1079, Train Error: 0.0413
Epoch: 15, Loss: 0.0865, Train Error: 0.0313
Epoch: 16, Loss: 0.0781, Train Error: 0.0284
Epoch: 17, Loss: 0.0582, Train Error: 0.0235
Epoch: 18, Loss: 0.0716, Train Error: 0.0277
Epoch: 19, Loss: 0.0601, Train Error: 0.0206
Epoch: 20, Loss: 0.0814, Train Error: 0.0277
Start Testing

True Bag Label: [0.]  Predicted Bag Label: [0.]


## PROSTATE + BLADDER

In [ ]:
# Jupyter Notebook version of main.py

import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
from torch.utils.data import random_split, ConcatDataset, DataLoader

# Import your dataset and models
from dataloader import HistopathologyDataset
from model import Attention, GatedAttention

# ---------------------------
# Set parameters manually:
# ---------------------------
epochs = 20
lr = 0.0005
reg = 1e-4
# Use your actual paths to the folder (or file) containing your .pt embeddings 
# and the CSV file with labels.
# (Note: if your HistopathologyDataset expects a directory, make sure you supply that.)
# Paths for Cancer Prostate
embeddings_dir_prostate = "/data/temporary/amirhosein/model_script_prostate/pt_files/"
labels_csv_prostate = "/data/temporary/amirhosein/Prostate_Project/prostate_tp53_labels.csv"

# Paths for Cancer Bladder
embeddings_dir_bladder = "/data/temporary/amirhosein/model_script_bladder/pt_files/"
labels_csv_bladder = "/data/temporary/amirhosein/bladder_project/bladder_tp53_labels.csv"


input_dim = 1024
model_type = "attention"   # Choose either "attention" or "gated_attention"
no_cuda = False

cuda = (not no_cuda) and torch.cuda.is_available()

# ---------------------------
# Set seeds and print status:
# ---------------------------
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

# ---------------------------
# Create dataset objects.
# ---------------------------
dataset_prostate = HistopathologyDataset(embeddings_dir_prostate, labels_csv_prostate)
dataset_bladder = HistopathologyDataset(embeddings_dir_bladder, labels_csv_bladder)


# Suppose you want 80% of Cancer C for training and 20% for testing.
total_prostate = len(dataset_prostate)
train_size_prostate = int(0.8 * total_prostate)
test_size_prostate  = total_prostate - train_size_prostate

dataset_prostate_train, dataset_prostate_test = random_split(dataset_prostate, [train_size_prostate, test_size_prostate])

train_dataset = ConcatDataset([dataset_bladder, dataset_prostate_train])
test_dataset  = dataset_prostate_test  # held-out portion from Cancer C

loader_kwargs = {'num_workers': 0, 'pin_memory': True} if cuda else {}
train_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, **loader_kwargs)
test_loader  = data.DataLoader(test_dataset, batch_size=1, shuffle=False, **loader_kwargs)

# ---------------------------
# Initialize the Model:
# ---------------------------
print('Initializing Model')
if model_type == 'attention':
    model = Attention(input_dim=input_dim)
elif model_type == 'gated_attention':
    model = GatedAttention(input_dim=input_dim)
else:
    raise ValueError("Unknown model type. Choose 'attention' or 'gated_attention'.")

if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=reg)

# ---------------------------
# Define training and testing functions:
# ---------------------------
def train(epoch):
    model.train()
    train_loss = 0.0
    train_error = 0.0
    for batch_idx, (data_batch, label) in enumerate(train_loader):
        if cuda:
            data_batch, label = data_batch.cuda(), label.cuda()
        data_batch, label = Variable(data_batch), Variable(label)
    
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data_batch, label)
        train_loss += loss.item()
        error, _ = model.calculate_classification_error(data_batch, label)
        train_error += error
    
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print('Epoch: {}, Loss: {:.4f}, Train Error: {:.4f}'.format(epoch, train_loss, train_error))
    
def test():
    model.eval()
    test_loss = 0.0
    test_error = 0.0
    with torch.no_grad():
        for batch_idx, (data_batch, label) in enumerate(test_loader):
            if cuda:
                data_batch, label = data_batch.cuda(), label.cuda()
            data_batch, label = Variable(data_batch), Variable(label)
            loss, attention_weights = model.calculate_objective(data_batch, label)
            test_loss += loss.item()
            error, predicted_label = model.calculate_classification_error(data_batch, label)
            test_error += error
    
            if batch_idx < 5:
                print('\nTrue Bag Label: {}  Predicted Bag Label: {}'
                      .format(label.cpu().numpy()[0], predicted_label.cpu().numpy()[0]))
    
    test_loss /= len(test_loader)
    test_error /= len(test_loader)
    print('\nTest Set, Loss: {:.4f}, Test Error: {:.4f}'.format(test_loss, test_error))
    
# ---------------------------
# Run training and testing:
# ---------------------------
print('Start Training')
for epoch in range(1, epochs + 1):
    train(epoch)
print('Start Testing')
test()


GPU is ON!
Initializing Model
Start Training
Epoch: 1, Loss: 0.5515, Train Error: 0.2566
Epoch: 2, Loss: 0.4559, Train Error: 0.2087
Epoch: 3, Loss: 0.4071, Train Error: 0.1793
Epoch: 4, Loss: 0.3423, Train Error: 0.1546


## Prostate + Breast

In [ ]:
# Jupyter Notebook version of main.py

import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
from torch.utils.data import random_split, ConcatDataset, DataLoader

# Import your dataset and models
from dataloader import HistopathologyDataset
from model import Attention, GatedAttention

# ---------------------------
# Set parameters manually:
# ---------------------------
epochs = 20
lr = 0.0005
reg = 1e-4
# Use your actual paths to the folder (or file) containing your .pt embeddings 
# and the CSV file with labels.
# (Note: if your HistopathologyDataset expects a directory, make sure you supply that.)
# Paths for Cancer Prostate
embeddings_dir_prostate = "/data/temporary/amirhosein/model_script_prostate/pt_files/"
labels_csv_prostate = "/data/temporary/amirhosein/Prostate_Project/prostate_tp53_labels.csv"

# Paths for Cancer Breast
embeddings_dir_breast = "/data/temporary/amirhosein/modelscript_breast/pt_files/"
labels_csv_breast = "/data/temporary/amirhosein/breast_Project(subdirectories)/breast_tp53_labels.csv"

input_dim = 1024
model_type = "attention"   # Choose either "attention" or "gated_attention"
no_cuda = False

cuda = (not no_cuda) and torch.cuda.is_available()

# ---------------------------
# Set seeds and print status:
# ---------------------------
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

# ---------------------------
# Create dataset objects.
# ---------------------------
dataset_prostate = HistopathologyDataset(embeddings_dir_prostate, labels_csv_prostate)
dataset_breast = HistopathologyDataset(embeddings_dir_breast, labels_csv_breast)


# Suppose you want 80% of Cancer C for training and 20% for testing.
total_prostate = len(dataset_prostate)
train_size_prostate = int(0.8 * total_prostate)
test_size_prostate  = total_prostate - train_size_prostate

dataset_prostate_train, dataset_prostate_test = random_split(dataset_prostate, [train_size_prostate, test_size_prostate])

train_dataset = ConcatDataset([dataset_breast, dataset_prostate_train])
test_dataset  = dataset_prostate_test  # held-out portion from Cancer C

loader_kwargs = {'num_workers': 0, 'pin_memory': True} if cuda else {}
train_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, **loader_kwargs)
test_loader  = data.DataLoader(test_dataset, batch_size=1, shuffle=False, **loader_kwargs)

# ---------------------------
# Initialize the Model:
# ---------------------------
print('Initializing Model')
if model_type == 'attention':
    model = Attention(input_dim=input_dim)
elif model_type == 'gated_attention':
    model = GatedAttention(input_dim=input_dim)
else:
    raise ValueError("Unknown model type. Choose 'attention' or 'gated_attention'.")

if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=reg)

# ---------------------------
# Define training and testing functions:
# ---------------------------
def train(epoch):
    model.train()
    train_loss = 0.0
    train_error = 0.0
    for batch_idx, (data_batch, label) in enumerate(train_loader):
        if cuda:
            data_batch, label = data_batch.cuda(), label.cuda()
        data_batch, label = Variable(data_batch), Variable(label)
    
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data_batch, label)
        train_loss += loss.item()
        error, _ = model.calculate_classification_error(data_batch, label)
        train_error += error
    
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print('Epoch: {}, Loss: {:.4f}, Train Error: {:.4f}'.format(epoch, train_loss, train_error))
    
def test():
    model.eval()
    test_loss = 0.0
    test_error = 0.0
    with torch.no_grad():
        for batch_idx, (data_batch, label) in enumerate(test_loader):
            if cuda:
                data_batch, label = data_batch.cuda(), label.cuda()
            data_batch, label = Variable(data_batch), Variable(label)
            loss, attention_weights = model.calculate_objective(data_batch, label)
            test_loss += loss.item()
            error, predicted_label = model.calculate_classification_error(data_batch, label)
            test_error += error
    
            if batch_idx < 5:
                print('\nTrue Bag Label: {}  Predicted Bag Label: {}'
                      .format(label.cpu().numpy()[0], predicted_label.cpu().numpy()[0]))
    
    test_loss /= len(test_loader)
    test_error /= len(test_loader)
    print('\nTest Set, Loss: {:.4f}, Test Error: {:.4f}'.format(test_loss, test_error))
    
# ---------------------------
# Run training and testing:
# ---------------------------
print('Start Training')
for epoch in range(1, epochs + 1):
    train(epoch)
print('Start Testing')
test()

## PROSTATE

In [ ]:
# Jupyter Notebook version of main.py

import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable

# Import your dataset and models
from dataloader import HistopathologyDataset
from model import Attention, GatedAttention

# ---------------------------
# Set parameters manually:
# ---------------------------
epochs = 20
lr = 0.0005
reg = 1e-4
# Use your actual paths to the folder (or file) containing your .pt embeddings 
# and the CSV file with labels.
# (Note: if your HistopathologyDataset expects a directory, make sure you supply that.)
embeddings_dir = "/data/temporary/amirhosein/model_script_prostate/pt_files/"
labels_csv = "/data/temporary/amirhosein/Prostate_Project/prostate_tp53_labels.csv"
input_dim = 1024
model_type = "attention"   # Choose either "attention" or "gated_attention"
no_cuda = False

cuda = (not no_cuda) and torch.cuda.is_available()

# ---------------------------
# Set seeds and print status:
# ---------------------------
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

# ---------------------------
# Create dataset objects.
# ---------------------------
train_dataset = HistopathologyDataset(embeddings_dir, labels_csv)
test_dataset  = HistopathologyDataset(embeddings_dir, labels_csv)

loader_kwargs = {'num_workers': 0, 'pin_memory': True} if cuda else {}
train_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, **loader_kwargs)
test_loader  = data.DataLoader(test_dataset, batch_size=1, shuffle=False, **loader_kwargs)

# ---------------------------
# Initialize the Model:
# ---------------------------
print('Initializing Model')
if model_type == 'attention':
    model = Attention(input_dim=input_dim)
elif model_type == 'gated_attention':
    model = GatedAttention(input_dim=input_dim)
else:
    raise ValueError("Unknown model type. Choose 'attention' or 'gated_attention'.")

if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=reg)

# ---------------------------
# Define training and testing functions:
# ---------------------------
def train(epoch):
    model.train()
    train_loss = 0.0
    train_error = 0.0
    for batch_idx, (data_batch, label) in enumerate(train_loader):
        if cuda:
            data_batch, label = data_batch.cuda(), label.cuda()
        data_batch, label = Variable(data_batch), Variable(label)
    
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data_batch, label)
        train_loss += loss.item()
        error, _ = model.calculate_classification_error(data_batch, label)
        train_error += error
    
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print('Epoch: {}, Loss: {:.4f}, Train Error: {:.4f}'.format(epoch, train_loss, train_error))
    
def test():
    model.eval()
    test_loss = 0.0
    test_error = 0.0
    with torch.no_grad():
        for batch_idx, (data_batch, label) in enumerate(test_loader):
            if cuda:
                data_batch, label = data_batch.cuda(), label.cuda()
            data_batch, label = Variable(data_batch), Variable(label)
            loss, attention_weights = model.calculate_objective(data_batch, label)
            test_loss += loss.item()
            error, predicted_label = model.calculate_classification_error(data_batch, label)
            test_error += error
    
            if batch_idx < 5:
                print('\nTrue Bag Label: {}  Predicted Bag Label: {}'
                      .format(label.cpu().numpy()[0], predicted_label.cpu().numpy()[0]))
    
    test_loss /= len(test_loader)
    test_error /= len(test_loader)
    print('\nTest Set, Loss: {:.4f}, Test Error: {:.4f}'.format(test_loss, test_error))
    
# ---------------------------
# Run training and testing:
# ---------------------------
print('Start Training')
for epoch in range(1, epochs + 1):
    train(epoch)
print('Start Testing')
test()


# original CNN 

## PROSTATE + BLADDER + BREAST 

# Jupyter Notebook version of main.py

import numpy as np
import torch
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
from torch.utils.data import random_split, ConcatDataset, DataLoader

# Import your dataset and models
from dataloader import HistopathologyDataset
from model_2 import Attention, GatedAttention

# ---------------------------
# Set parameters manually:
# ---------------------------
epochs = 20
lr = 0.0005
reg = 1e-4
# Use your actual paths to the folder (or file) containing your .pt embeddings 
# and the CSV file with labels.
# (Note: if your HistopathologyDataset expects a directory, make sure you supply that.)
# Paths for Cancer Prostate
embeddings_dir_prostate = "/data/temporary/amirhosein/model_script_prostate/pt_files/"
labels_csv_prostate = "/data/temporary/amirhosein/Prostate_Project/prostate_tp53_labels.csv"

# Paths for Cancer Bladder
embeddings_dir_bladder = "/data/temporary/amirhosein/model_script_bladder/pt_files/"
labels_csv_bladder = "/data/temporary/amirhosein/bladder_project/bladder_tp53_labels.csv"

# Paths for Cancer Breast
embeddings_dir_breast = "/data/temporary/amirhosein/modelscript_breast/pt_files/"
labels_csv_breast = "/data/temporary/amirhosein/breast_Project(subdirectories)/breast_tp53_labels.csv"

input_dim = 1024
model_type = "attention"   # Choose either "attention" or "gated_attention"
no_cuda = False

cuda = (not no_cuda) and torch.cuda.is_available()

# ---------------------------
# Set seeds and print status:
# ---------------------------
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

# ---------------------------
# Create dataset objects.
# ---------------------------
dataset_prostate = HistopathologyDataset(embeddings_dir_prostate, labels_csv_prostate)
dataset_bladder = HistopathologyDataset(embeddings_dir_bladder, labels_csv_bladder)
dataset_breast = HistopathologyDataset(embeddings_dir_breast, labels_csv_breast)


# Suppose you want 80% of Cancer C for training and 20% for testing.
total_prostate = len(dataset_prostate)
train_size_prostate = int(0.8 * total_prostate)
test_size_prostate  = total_prostate - train_size_prostate

dataset_prostate_train, dataset_prostate_test = random_split(dataset_prostate, [train_size_prostate, test_size_prostate])

train_dataset = ConcatDataset([dataset_breast, dataset_bladder, dataset_prostate_train])
test_dataset  = dataset_prostate_test  # held-out portion from Cancer C

loader_kwargs = {'num_workers': 0, 'pin_memory': True} if cuda else {}
train_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, **loader_kwargs)
test_loader  = data.DataLoader(test_dataset, batch_size=1, shuffle=False, **loader_kwargs)

# ---------------------------
# Initialize the Model:
# ---------------------------
print('Initializing Model')
if model_type == 'attention':
    model = Attention()
elif model_type == 'gated_attention':
    model = GatedAttention()
else:
    raise ValueError("Unknown model type. Choose 'attention' or 'gated_attention'.")

if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=reg)

# ---------------------------
# Define training and testing functions:
# ---------------------------
def train(epoch):
    model.train()
    train_loss = 0.0
    train_error = 0.0
    for batch_idx, (data_batch, label) in enumerate(train_loader):
        if cuda:
            data_batch, label = data_batch.cuda(), label.cuda()
        data_batch, label = Variable(data_batch), Variable(label)
    
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data_batch, label)
        train_loss += loss.item()
        error, _ = model.calculate_classification_error(data_batch, label)
        train_error += error
    
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print('Epoch: {}, Loss: {:.4f}, Train Error: {:.4f}'.format(epoch, train_loss, train_error))
    
def test():
    model.eval()
    test_loss = 0.0
    test_error = 0.0
    with torch.no_grad():
        for batch_idx, (data_batch, label) in enumerate(test_loader):
            if cuda:
                data_batch, label = data_batch.cuda(), label.cuda()
            data_batch, label = Variable(data_batch), Variable(label)
            loss, attention_weights = model.calculate_objective(data_batch, label)
            test_loss += loss.item()
            error, predicted_label = model.calculate_classification_error(data_batch, label)
            test_error += error
    
            if batch_idx < 5:
                print('\nTrue Bag Label: {}  Predicted Bag Label: {}'
                      .format(label.cpu().numpy()[0], predicted_label.cpu().numpy()[0]))
    
    test_loss /= len(test_loader)
    test_error /= len(test_loader)
    print('\nTest Set, Loss: {:.4f}, Test Error: {:.4f}'.format(test_loss, test_error))
    
# ---------------------------
# Run training and testing:
# ---------------------------
print('Start Training')
for epoch in range(1, epochs + 1):
    train(epoch)
print('Start Testing')
test()